In [7]:
import pandas as pd
import numpy as np
df = pd.read_csv('./data/real_estate_raw.csv', encoding = 'utf-8')

df.columns = ['鄉鎮市區',
              '交易標的',
              '土地區段位置建物區段門牌',
              '土地移轉總面積平方公尺',
              '都市土地使用分區',
              '非都市土地使用分區',
              '非都市土地使用編定',
              '交易年月日',
              '交易筆棟數',
              '移轉層次',
              '總樓層數',
              '建物型態',
              '主要用途',
              '主要建材',
              '建築完成年月',
              '建物移轉總面積平方公尺',
              '建物現況格局-房',
              '建物現況格局-廳',
              '建物現況格局-衛',
              '建物現況格局-隔間',
              '有無管理組織',
              '總價元',
              '單價元平方公尺',
              '車位類別',
              '車位移轉總面積平方公尺',
              '車位總價元',
              '備註',
              '編號',
              '主建物面積',
              '附屬建物面積',
              '陽台面積',
              '電梯']

print(df)
print(df.dtypes)


# 找出'單價元平方公尺'的缺失值為備註欄中包含'僅車位交易'的欄位
print(df.isnull().sum())
print(df[df['單價元平方公尺'].isnull()])


# 找出若是'交易標的'欄位為'土地'或'車位'的話，'建物型態'就會標註為'其他'，故'建物型態'的缺失值為內容為'其他'的欄位
print(df['建物型態'])


# 找出'都市土地使用分區'的缺失值為第0筆為'都市：其他：人行步道'，需刪除。
print(df['都市土地使用分區'])



from datetime import datetime

# 將'交易年月日'及'建築完成年月'欄位從民國年轉西元年
df['交易年月日'] = pd.to_datetime(df['交易年月日'] + 19110000, format = '%Y%m%d')
df['建築完成年月'] = pd.to_datetime(df['建築完成年月'] + 19110000, format = '%Y%m%d')




# 刪除'單價元平方公尺'中包含'僅車位交易'的欄位
a = df[~df['單價元平方公尺'].isnull()]

# 刪除'交易標的'欄位中包含'其他'字串的欄位
b = a[~a['建物型態'].isin(['其他'])]


# 刪除'備註'中含有'親友、員工、共有人或其他特殊關係間之交易'字串的欄位
content = '親友、員工、共有人或其他特殊關係間之交易'
c = b[~b.apply(lambda row: row.astype(str).str.contains(content).any(), axis = 1)]


# 刪除'備註'中含有'政府機關標讓售'字串的欄位
cont = '政府機關標讓售'
d = c[~c.apply(lambda row: row.astype(str).str.contains(cont).any(), axis = 1)]


# 刪除'備註'中含有'二親等間持分買賣'字串的欄位
con = '二親等間持分買賣'
e = d[~d.apply(lambda row: row.astype(str).str.contains(con).any(), axis = 1)]



# 將'單價元平方公尺欄位的格式改為int
print(e['單價元平方公尺'].astype('int64'))




# 將工程格式的數字轉換成浮點數，(若要轉成整數的話，就在結尾再多加'.astype('int64')')
print(e['總價元'].describe().astype('float64').astype('int64'))
print(e['單價元平方公尺'].astype('float64'))
print(e['建物移轉總面積平方公尺'].describe())

print(e.dtypes)
print(e.shape)

# 依'土地區段位置建物區段門牌'進行排序
road = e.sort_values(by = '土地區段位置建物區段門牌', ascending = True)
print(road)


add = e['土地區段位置建物區段門牌'].str.extract(r'([一-龥]+(?:路|街|大道))')
add.columns = ['道路名稱']
add_section = e['土地區段位置建物區段門牌'].str.extract(r'([一-龥]+路[一二三四五六七八九十]段)')
add_section.columns = ['段']

# 將以上欄位合併回原本的DataFrame中
con = pd.concat([e, add, add_section], axis = 1)
con['分類標籤'] = con['段'].fillna(con['道路名稱'])

# 計算均價
grouped = con.groupby('分類標籤')['單價元平方公尺'].mean().reset_index()
print(grouped)


# 將路名進行分類排序
pd.set_option('display.float_format', lambda x: '%.5f' % x)
# con.groupby('道路名稱').agg({'總價元':'mean'})

print(con[['總價元', '單價元平方公尺']].describe())

# geocoding
To_list = con['土地區段位置建物區段門牌'].tolist()

# con['土地區段位置建物區段門牌'].to_csv('./data/地址.csv', index = 0, header = 1, encoding = 'utf-8')

'''
可成功，但為了不浪費試用額，故以備註呈現，並將結果另存成地址經緯度.csv檔。

用dotenv將api key隱藏在系統變數中
from dotenv import load_dotenv
load_dotenv()
import os
load_key = os.environ.get('GOOGLE_MAP_API_Key')

中和實價登錄geocoding
import googlemaps
import time

gmaps = googlemaps.Client(key = load_key)

address = To_list

ids = []
for addr in address:
    geocode_result = gmaps.geocode(addr)
    loc = geocode_result[0]['geometry']['location']
    time.sleep(0.1)
    print(loc)
'''  

# con.to_csv('./data/real_estate_cleaned.csv', index = 0, header = 1, encoding = 'utf-8')



     鄉鎮市區       交易標的             土地區段位置建物區段門牌  土地移轉總面積平方公尺    都市土地使用分區  \
0     中和區         土地                 錦和段316地號     14.40000  都市：其他:人行步道   
1     中和區  房地(土地+建物)  新北市中和區興南路一段１０５巷２０弄１７號三樓     12.87000           住   
2     中和區  房地(土地+建物)       新北市中和區國光街１０２巷２７號四樓     16.98000           住   
3     中和區  房地(土地+建物)          新北市中和區大勇街５７號十二樓     10.80000           住   
4     中和區         車位      新北市中和區南華路６７、６９號十三樓等      0.75000           住   
...   ...        ...                      ...          ...         ...   
4515  中和區  房地(土地+建物)         新北市中和區和平街６８號四樓之２      8.46000           住   
4516  中和區  房地(土地+建物)         新北市中和區和平街６８號七樓之３      8.46000           住   
4517  中和區  房地(土地+建物)         新北市中和區和平街６８號六樓之６     12.32000           住   
4518  中和區  房地(土地+建物)         新北市中和區和平街６８號六樓之３      8.46000           住   
4519  中和區  房地(土地+建物)         新北市中和區和平街６８號九樓之８      8.66000           住   

      非都市土地使用分區  非都市土地使用編定    交易年月日      交易筆棟數   移轉層次  ...      單價元平方公尺  車位類別  \
0           NaN        NaN  11